# Hour

13 - 24시를 12 - 0시로 바꿔서 순환하는 느낌?으로 해보려했는데 전혀 상관 없었다.

### Overview
** Model **
  * BernoulliNB. All hyperparameters are default.
  
** Features **
  * X, Y
  * Dates (Convert to numerical columns)
    * Conver the Dates column to numerical columns named Dates-Year, Dates-Month, Dates-Day, Dates-Hour and Dates-Minute.
    * Modify Dates-Minute to zero if the value is equal to 30.
    * Use only Dates-Hour and Dates-Minute.


### Result
  * 5-fold Cross Validation = **2.589878**
  * Leaderboard = **2.59008**

In [1]:
import numpy as np
import pandas as pd

## Load Data

In [2]:
train = pd.read_csv("../data/train.csv")
train.head(3)

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414


In [3]:
test = pd.read_csv("../data/test.csv")
test.head(3)

,Id,Dates,DayOfWeek,PdDistrict,Address,X,Y
0,0,2015-05-10 23:59:00,Sunday,BAYVIEW,2000 Block of THOMAS AV,-122.399588,37.735051
1,1,2015-05-10 23:51:00,Sunday,BAYVIEW,3RD ST / REVERE AV,-122.391523,37.732432
2,2,2015-05-10 23:50:00,Sunday,NORTHERN,2000 Block of GOUGH ST,-122.426002,37.792212


## Feature Extraction

### Merge both the train and the test dataframe

In [4]:
seperator = train.shape[0]

train["combi-index"] = ["train-{0}".format(index) for index, _ in train.iterrows()]
test["combi-index"] = ["test-{0}".format(index) for index, _ in test.iterrows()]

combi = pd.concat([train, test])
combi = combi.set_index("combi-index")

combi.head(3)

,Address,Category,Dates,DayOfWeek,Descript,Id,PdDistrict,Resolution,X,Y
combi-index,,,,,,,,,,
train-0,OAK ST / LAGUNA ST,WARRANTS,2015-05-13 23:53:00,Wednesday,WARRANT ARREST,NaN,NORTHERN,"ARREST, BOOKED",-122.425892,37.774599
train-1,OAK ST / LAGUNA ST,OTHER OFFENSES,2015-05-13 23:53:00,Wednesday,TRAFFIC VIOLATION ARREST,NaN,NORTHERN,"ARREST, BOOKED",-122.425892,37.774599
train-2,VANNESS AV / GREENWICH ST,OTHER OFFENSES,2015-05-13 23:33:00,Wednesday,TRAFFIC VIOLATION ARREST,NaN,NORTHERN,"ARREST, BOOKED",-122.424363,37.800414


### Convert the 'Dates' column to numerical columns


In [5]:
from datetime import datetime

total_count = combi.shape[0]
count = 0

dates_data = []

for index, row in combi["Dates"].iteritems():
    count = count + 1

    if count % 100000 == 0:
        print("processing... {0}/{1}".format(count, total_count))

    date = datetime.strptime(row, "%Y-%m-%d %H:%M:%S")

    dates_data.append({
        "combi-index": index,
        "Dates-Year": date.year,
        "Dates-Month": date.month,
        "Dates-Day": date.day,
        "Dates-Hour": date.hour if date.hour <= 12 else 24 - date.hour,
        "Dates-Minute": date.minute,
        "Dates-Second": date.second,
    })
    
dates_dataframe = pd.DataFrame.from_dict(dates_data)
dates_dataframe = dates_dataframe.set_index("combi-index")

dates_columns = ["Dates-Year", "Dates-Month", "Dates-Day", "Dates-Hour", "Dates-Minute", "Dates-Second"]
dates_dataframe = dates_dataframe[dates_columns]

# All "Dates-Second" variable is equal to zero. Therefore, we can remove it.
second_list = dates_dataframe["Dates-Second"].unique()
print("list of seconds = {0}".format(second_list))

dates_dataframe = dates_dataframe.drop("Dates-Second", axis=1)

combi = pd.concat([combi, dates_dataframe], axis=1)

combi.head(3)

processing... 100000/1762311
processing... 200000/1762311
processing... 300000/1762311
processing... 400000/1762311
processing... 500000/1762311
processing... 600000/1762311
processing... 700000/1762311
processing... 800000/1762311
processing... 900000/1762311
processing... 1000000/1762311
processing... 1100000/1762311
processing... 1200000/1762311
processing... 1300000/1762311
processing... 1400000/1762311
processing... 1500000/1762311
processing... 1600000/1762311
processing... 1700000/1762311
list of seconds = [0]


,Address,Category,Dates,DayOfWeek,Descript,Id,PdDistrict,Resolution,X,Y,Dates-Year,Dates-Month,Dates-Day,Dates-Hour,Dates-Minute
combi-index,,,,,,,,,,,,,,,
train-0,OAK ST / LAGUNA ST,WARRANTS,2015-05-13 23:53:00,Wednesday,WARRANT ARREST,NaN,NORTHERN,"ARREST, BOOKED",-122.425892,37.774599,2015,5,13,1,53
train-1,OAK ST / LAGUNA ST,OTHER OFFENSES,2015-05-13 23:53:00,Wednesday,TRAFFIC VIOLATION ARREST,NaN,NORTHERN,"ARREST, BOOKED",-122.425892,37.774599,2015,5,13,1,53
train-2,VANNESS AV / GREENWICH ST,OTHER OFFENSES,2015-05-13 23:33:00,Wednesday,TRAFFIC VIOLATION ARREST,NaN,NORTHERN,"ARREST, BOOKED",-122.424363,37.800414,2015,5,13,1,33


### Modify the **Dates-Minute** to 0 if the value is 30

In [6]:
combi.loc[combi["Dates-Minute"] == 30, "Dates-Minute"] = 0
print("The number of rows which the Date-Minutes is equal to 30 = {0}".format(combi[combi["Dates-Minute"] == 30].shape[0]))

The number of rows which the Date-Minutes is equal to 30 = 0


### Split to the train and the test dataframe

In [7]:
train = combi[:seperator]
train = train.drop("Id", axis=1)
train.head(3)

,Address,Category,Dates,DayOfWeek,Descript,PdDistrict,Resolution,X,Y,Dates-Year,Dates-Month,Dates-Day,Dates-Hour,Dates-Minute
combi-index,,,,,,,,,,,,,,
train-0,OAK ST / LAGUNA ST,WARRANTS,2015-05-13 23:53:00,Wednesday,WARRANT ARREST,NORTHERN,"ARREST, BOOKED",-122.425892,37.774599,2015,5,13,1,53
train-1,OAK ST / LAGUNA ST,OTHER OFFENSES,2015-05-13 23:53:00,Wednesday,TRAFFIC VIOLATION ARREST,NORTHERN,"ARREST, BOOKED",-122.425892,37.774599,2015,5,13,1,53
train-2,VANNESS AV / GREENWICH ST,OTHER OFFENSES,2015-05-13 23:33:00,Wednesday,TRAFFIC VIOLATION ARREST,NORTHERN,"ARREST, BOOKED",-122.424363,37.800414,2015,5,13,1,33


In [8]:
test = combi[seperator:]
test = test.drop(["Category", "Descript", "Resolution"], axis=1)
test["Id"] = test["Id"].astype('int32')

test = test.set_index(["Id"])
test.head(3)

,Address,Dates,DayOfWeek,PdDistrict,X,Y,Dates-Year,Dates-Month,Dates-Day,Dates-Hour,Dates-Minute
Id,,,,,,,,,,,
0,2000 Block of THOMAS AV,2015-05-10 23:59:00,Sunday,BAYVIEW,-122.399588,37.735051,2015,5,10,1,59
1,3RD ST / REVERE AV,2015-05-10 23:51:00,Sunday,BAYVIEW,-122.391523,37.732432,2015,5,10,1,51
2,2000 Block of GOUGH ST,2015-05-10 23:50:00,Sunday,NORTHERN,-122.426002,37.792212,2015,5,10,1,50


## Score

In [9]:
from sklearn.cross_validation import cross_val_score

feature_names = ["X", "Y"] + ["Dates-Hour", "Dates-Minute"]
label_name = "Category"

train_X = train[feature_names]
test_X = test[feature_names]

train_y = train[label_name]

train_X.head(3)

,X,Y,Dates-Hour,Dates-Minute
combi-index,,,,
train-0,-122.425892,37.774599,1,53
train-1,-122.425892,37.774599,1,53
train-2,-122.424363,37.800414,1,33


In [10]:
from sklearn.naive_bayes import BernoulliNB

model = BernoulliNB()
score = cross_val_score(model, train_X, train_y, scoring='log_loss', cv=5).mean()
print("BernoulliNB = {0:.6f}".format(-1.0 * score))

BernoulliNB = 2.589878


## Prediction

In [11]:
model = BernoulliNB()
model.fit(train_X, train_y)

prediction = model.predict_proba(test_X)
prediction[0:100]

array([[ 0.00217153,  0.10014093,  0.00023882, ...,  0.02337961,
         0.07587894,  0.01437121],
       [ 0.00217153,  0.10014093,  0.00023882, ...,  0.02337961,
         0.07587894,  0.01437121],
       [ 0.00217153,  0.10014093,  0.00023882, ...,  0.02337961,
         0.07587894,  0.01437121],
       ..., 
       [ 0.00217153,  0.10014093,  0.00023882, ...,  0.02337961,
         0.07587894,  0.01437121],
       [ 0.00217153,  0.10014093,  0.00023882, ...,  0.02337961,
         0.07587894,  0.01437121],
       [ 0.00217153,  0.10014093,  0.00023882, ...,  0.02337961,
         0.07587894,  0.01437121]])

# Submission

In [12]:
sample = pd.read_csv("../data/sampleSubmission.csv", index_col="Id")
sample.head(3)

,ARSON,ASSAULT,BAD CHECKS,BRIBERY,BURGLARY,DISORDERLY CONDUCT,DRIVING UNDER THE INFLUENCE,DRUG/NARCOTIC,DRUNKENNESS,EMBEZZLEMENT,...,SEX OFFENSES NON FORCIBLE,STOLEN PROPERTY,SUICIDE,SUSPICIOUS OCC,TREA,TRESPASS,VANDALISM,VEHICLE THEFT,WARRANTS,WEAPON LAWS
Id,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [13]:
submission = pd.DataFrame(prediction, index=sample.index)
submission.columns = sample.columns
submission.head()

,ARSON,ASSAULT,BAD CHECKS,BRIBERY,BURGLARY,DISORDERLY CONDUCT,DRIVING UNDER THE INFLUENCE,DRUG/NARCOTIC,DRUNKENNESS,EMBEZZLEMENT,...,SEX OFFENSES NON FORCIBLE,STOLEN PROPERTY,SUICIDE,SUSPICIOUS OCC,TREA,TRESPASS,VANDALISM,VEHICLE THEFT,WARRANTS,WEAPON LAWS
Id,,,,,,,,,,,,,,,,,,,,,
0,0.002172,0.100141,0.000239,0.000364,0.026684,0.006924,0.00393,0.099098,0.007451,0.000846,...,0.0001,0.007094,0.000698,0.038186,0.000008,0.011423,0.037386,0.02338,0.075879,0.014371
1,0.002172,0.100141,0.000239,0.000364,0.026684,0.006924,0.00393,0.099098,0.007451,0.000846,...,0.0001,0.007094,0.000698,0.038186,0.000008,0.011423,0.037386,0.02338,0.075879,0.014371
2,0.002172,0.100141,0.000239,0.000364,0.026684,0.006924,0.00393,0.099098,0.007451,0.000846,...,0.0001,0.007094,0.000698,0.038186,0.000008,0.011423,0.037386,0.02338,0.075879,0.014371
3,0.002172,0.100141,0.000239,0.000364,0.026684,0.006924,0.00393,0.099098,0.007451,0.000846,...,0.0001,0.007094,0.000698,0.038186,0.000008,0.011423,0.037386,0.02338,0.075879,0.014371
4,0.002172,0.100141,0.000239,0.000364,0.026684,0.006924,0.00393,0.099098,0.007451,0.000846,...,0.0001,0.007094,0.000698,0.038186,0.000008,0.011423,0.037386,0.02338,0.075879,0.014371


In [14]:
from datetime import datetime

current_time = datetime.now().strftime("%Y.%m.%d %H:%M:%S")
description = "Curcular hour"

filename = "../submit/{0} {1}.csv".format(current_time, description)

submission.to_csv(filename)